In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from flarestack import analyse, ResultsHandler
from flarestack.data.simulate.potemkin import potemkin_dataset, IdealBackgroundFluxModel
from flarestack.utils.prepare_catalogue import ps_catalogue_name
from flarestack.data.public import icecube_ps_3_year
from flarestack.shared import flux_to_k
import logging
logging.getLogger().setLevel("INFO")

In [2]:
# We create an imaginary dataset season, between 50000 MJD and 55000 MJD, with 100% uptime

bkg_time_pdf_dict = {
    "time_pdf_name": "fixed_ref_box",
    "fixed_ref_time_mjd": 50000,
    "pre_window": 0.,
    "post_window": 500.,
    "livetime": True
}

# We take the rough normalisation of the atmospheric flux, ~10^8 GeV cm^-2 s^-1 sr^-1 at ~200 TeV

f_200tev = (10. ** -8.)
flux_norm_1_gev = f_200tev * ((2.e5) ** 1.7)

bkg_flux_model = IdealBackgroundFluxModel(
    flux_norm_1_gev,
    bkg_time_pdf_dict=bkg_time_pdf_dict,
)

# We create a simulated dataset, containg only background, using the published effective areas for IceCube with 86 strings 2012

potemkin_dataset.set_sim_params(
    name="IC86-2012",
    bkg_flux_model=bkg_flux_model,
#     resimulate=True
)

INFO:root:Found existing simulation at /Users/thomasahrens/Desktop/IceCube/flarestack/scratch/flarestack__data/input/sim_datasets/SimCube_default/IC86-2012/10274270429.662216//50000.0_50500.0.npy


In [3]:
# Setup analysis, injecting an E^-2 source

inj_dict = {
    "injection_sig_time_pdf": {
        "time_pdf_name": "steady"
    },
    "injection_energy_pdf": {
        "energy_pdf_name": "power_law",
        "gamma": 3.7
    }
}

# Perform a "standard" analysis (fitting spectral index and ns)

llh_dict = {
    "llh_name": "spatial",
    "llh_sig_time_pdf": {
        "time_pdf_name": "steady"
    },
    "llh_bkg_time_pdf":  {
        "time_pdf_name": "steady"
    },
#     "llh_energy_pdf": {
#         "energy_pdf_name": "power_law"
#     }
}

# Analysis with "source catalogue" consisting of a single source at sindec=0.5

mh_dict = {
            "name": "examples/analysis_with_sim",
            "mh_name": "fixed_weights",
#             "dataset": potemkin_dataset.get_seasons("IC86-2012"),
            "dataset": icecube_ps_3_year.get_seasons("IC86-2012"),
#             "catalogue": ps_catalogue_name(0.5),
            "catalogue": "/Users/thomasahrens/Desktop/IceCube/sn-search/test_catalogue.npy",
            "inj_dict": inj_dict,
            "llh_dict": llh_dict,
            "n_steps": 5,
            "n_trials": 10,
            "scale": 800000.
        }

In [4]:
print(potemkin_dataset.get_single_season("IC86-2012"))

In [5]:
analyse(mh_dict, n_cpu=2, cluster=False)

rh = ResultsHandler(mh_dict)

INFO:flarestack.core.minimisation:Using 'spatial' LLH class
INFO:flarestack.core.injector:Initialising Injector for IC86-2012
INFO:flarestack.core.minimisation:Using 'spatial' LLH class
INFO:flarestack.core.multiprocess_wrapper:Added 140 trials to queue. Now processing.
INFO:flarestack.core.multiprocess_wrapper:140 tasks remaining.
INFO:flarestack.core.minimisation:Using 'spatial' LLH class
/Users/thomasahrens/Desktop/IceCube/flarestack/flarestack/core/llh.py:436: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  SoB_spacetime = np.array(SoB_spacetime)
/Users/thomasahrens/Desktop/IceCube/flarestack/flarestack/core/llh.py:436: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different 